In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from pybliometrics.scopus import ScopusSearch
from bs4 import BeautifulSoup
import requests, json
from docx import Document
from docx.shared import Inches

# my data

In [2]:
# my data

from my_apikeys import APIKeys
my_names = ["Lee, Jehyun", "Lee, J.", "Lee, J. H.", "Lee, Je Hyun"]

# including reversed name
my_names_reverse = []
for n in my_names:
    n_reverse = " ".join(n.split(", ")[1:-1] + [n.split(", ")[-1] + ","] + [n.split(", ")[0]])
    my_names_reverse.append(n_reverse)

my_names = my_names + my_names_reverse

# including without comma
my_names_nocomma = [n.replace(",", "") for n in my_names]
my_names = my_names + my_names_nocomma  


In [3]:
# subscriber (Institute: True. Home: False)
try:
    s_sample = ScopusSearch(f"DOI (10.1021/acsnano.9b08494)").results[0]
    subscriber=True
except:
    subscriber=False

print(f"subscriber={subscriber}")    

subscriber=True


In [4]:
df_CV = pd.read_excel("my_CV.xlsx")
df_CV.head()

,DOI,TITLE_Scopus
0,10.1021/acs.jpclett.1c01140,First-Principles-Based Machine-Learning Molecu...
1,10.1038/s41598-021-83315-9,Optimizing machine learning models for granula...
2,10.1038/srep31513,Resonant vortex-core reversal in magnetic nano...
3,10.1063/1.4919836,Temperature effect on vortex-core reversals in...
4,10.1038/srep11370,Resonantly exited precession motion of three-d...


In [5]:
def get_my_data(doi):
    # retrieve data
    s = ScopusSearch(f"DOI ({doi})").results[0]
    
    ### Affiliation
    # Affiliation ID
    afids = np.array(s.afid.split(";"))    
    # Affiliation name
    afnames = np.array(s.affilname.split(";"))
    # Affiliations
    afs = dict(zip(afids, afnames))
    afs_all.update(afs)
    
    ### Author
    # Author names
    author_names = np.array(s.author_names.split(";"))
    # Author ID
    author_ids = np.array(s.author_ids.split(";"))
    # Author Affiliation ID
    author_afids = [a.split("-") for a in np.array(s.author_afids.split(";"))]
    # Authors
    authors = list(zip(author_names, author_ids, [afs[af] if isinstance(af, str) else [afs[af_] for af_ in af] for af in author_afids], author_afids))
    
    try:
        # my data
        name_search = np.array([[author_names == my_name] for my_name in my_names]).sum(axis=0).flatten()
        my_idx = np.where(name_search > 0)[0][0]
        my_data = list(authors[my_idx])
        # coauthor
        coauthor_names = list(np.delete(author_names, my_idx))
        coauthor_ids = list(np.delete(author_ids, my_idx, 0))
        coauthor_data = [coauthor_names, coauthor_ids]        
        return my_data + coauthor_data
    except:
        print(authors)
        pass

In [6]:
afs_all = {}
my_data_all = []

for doi in df_CV["DOI"].values:
    data = get_my_data(doi)
    my_data_all.append(data)

In [7]:
df_my = pd.DataFrame(my_data_all, 
                     columns=["my_name", "my_id", "my_afs", "my_afids", "co_names", "co_ids"])
df_my.to_excel("my_data.xlsx", index=False)
df_my.head()

,my_name,my_id,my_afs,my_afids,co_names,co_ids
0,"Lee, Jehyun",55969873500,[Korea Institute of Energy Research],[60087822],"[Hong, Sung Jun, Chun, Hoje, Kim, Byung Hyun, ...","[57221590144, 57195466596, 55502655900, 572207..."
1,"Lee, Jehyun",55969873500,[Korea Institute of Energy Research],[60087822],"[Park, Hyeon Kyu, Lee, Jae Hyeok, Kim, Sang Koog]","[57216558093, 57206731799, 7601587994]"
2,"Lee, Jehyun",16029235100,"[Seoul National University, Center of Semicond...","[60013682, 111165432]","[Kim, Sang Koog, Yoo, Myoung Woo, Lee, Jae Hye...","[7601587994, 35738330300, 57206731799, 5719974..."
3,"Lee, Jehyun",16029235100,"[Seoul National University, Center of Semicond...","[60013682, 115225850]","[Kim, Bosung, Yoo, Myoung Woo, Kim, Sang Koog]","[56621593100, 35738330300, 7601587994]"
4,"Lee, Jehyun",16029235100,[National Creative Research Initiative Center ...,"[107172817, 115225850]","[Kim, Sang Koog, Yoo, Myoung Woo, Lee, Ha Youn...","[7601587994, 35738330300, 56206010000, 5720673..."


In [8]:
df_my = df_my.explode(["my_afs", "my_afids"]).explode(["co_names", "co_ids"])
df_my.drop_duplicates(inplace=True)
df_my

,my_name,my_id,my_afs,my_afids,co_names,co_ids
0,"Lee, Jehyun",55969873500,Korea Institute of Energy Research,60087822,"Hong, Sung Jun",57221590144
0,"Lee, Jehyun",55969873500,Korea Institute of Energy Research,60087822,"Chun, Hoje",57195466596
0,"Lee, Jehyun",55969873500,Korea Institute of Energy Research,60087822,"Kim, Byung Hyun",55502655900
0,"Lee, Jehyun",55969873500,Korea Institute of Energy Research,60087822,"Seo, Min Ho",57220796436
0,"Lee, Jehyun",55969873500,Korea Institute of Energy Research,60087822,"Kang, Joonhee",56076915900
...,...,...,...,...,...,...
43,"Lee, Je Hyun",16029235100,Korea Institute of Science and Technology,60025960,"Kim, Jang Hee",37106931200
43,"Lee, Je Hyun",16029235100,Korea Institute of Science and Technology,60025960,"Chang, Joonyeon",57203369778
43,"Lee, Je Hyun",16029235100,Korea Institute of Science and Technology,60025960,"Kim, Hi Jung",57191720425
43,"Lee, Je Hyun",16029235100,Korea Institute of Science and Technology,60025960,"Kim, Inyoung",7404144290


In [9]:
df_my = df_my.groupby(["my_name", "my_id", "my_afs", "my_afids"]).agg(list).reset_index()
df_my

,my_name,my_id,my_afs,my_afids,co_names,co_ids
0,"Jehyun, Lee",15834698800,Seoul National University,60013682,"[Suess, Dieter, Schrefl, Thomas, Kyu, Hwan Oh,...","[7004076065, 7005780657, 6602521869, 35601590600]"
1,"Jehyun, Lee",15834698800,Technische Universität Wien,60018163,"[Suess, Dieter, Schrefl, Thomas, Kyu, Hwan Oh,...","[7004076065, 7005780657, 6602521869, 35601590600]"
2,"Lee, J.",16029235100,Technische Universität Wien,60018163,"[Fiedler, G., Fidler, J., Schrefl, T., Stamps,...","[15829081900, 35601590600, 7005780657, 7004916..."
3,"Lee, J.",24399437900,Technische Universität Wien,60018163,"[Schulze, C., Faustini, M., Schletter, H., Lut...","[35181915100, 57196395997, 36675826300, 350853..."
4,"Lee, J.",7601454858,Seoul National University,60013682,"[Neu, V., Schulze, C., Faustini, M., Makarov, ...","[6701391076, 35181915100, 57196395997, 1264713..."
5,"Lee, J. H.",7601451606,Seoul National University,60013682,"[Jang, S. H., Kim, Y. W., Kim, K. Y.]","[24821314300, 8099600200, 57203256286]"
6,"Lee, J. H.",7601462370,Korea Institute of Science and Technology,60025960,"[Jang, S. H., Kang, T., Kim, K. Y.]","[24821314300, 8953533300, 57203256286]"
7,"Lee, Je Hyun",16029235100,Korea Institute of Science and Technology,60025960,"[Kim, Dal Young, Kim, Jin Soo, Park, Bae Ho, L...","[7409765949, 55881881500, 7402834739, 56524680..."
8,"Lee, Jehyun",16029235100,Center of Semiconductor Research and Development,111165432,"[Kim, Sang Koog, Yoo, Myoung Woo, Lee, Jae Hye...","[7601587994, 35738330300, 57206731799, 5719974..."
9,"Lee, Jehyun",16029235100,Center of Semiconductor Research and Development,115225850,"[Kim, Bosung, Yoo, Myoung Woo, Kim, Sang Koog,...","[56621593100, 35738330300, 7601587994, 5620601..."


In [10]:
df_my.describe()

,my_name,my_id,my_afs,my_afids,co_names,co_ids
count,15,15,15,15,15,15
unique,5,7,7,8,14,14
top,"Lee, Jehyun",16029235100,Seoul National University,60013682,"[Suess, Dieter, Schrefl, Thomas, Kyu, Hwan Oh,...","[7004076065, 7005780657, 6602521869, 35601590600]"
freq,7,8,4,4,2,2


# my paper

In [11]:
# def get_scopus_info(SCOPUS_ID):
#     url = ("http://api.elsevier.com/content/abstract/scopus_id/"
#           + SCOPUS_ID
#           + "?field=authors,title,publicationName,volume,issueIdentifier,"
#           + "prism:pageRange,coverDate,article-number,doi,citedby-count,prism:aggregationType")
#     resp = requests.get(url,
#                     headers={'Accept':'application/json',
#                              'X-ELS-APIKey': APIKeys[-3]})
#     results = json.loads(resp.text.encode('utf-8'))
    
#     authors = ', '.join([au['ce:indexed-name'] for au in results['abstracts-retrieval-response']['authors']['author']])
#     title = results['abstracts-retrieval-response']['coredata']['dc:title']
#     pubtype = results['abstracts-retrieval-response']['coredata']['prism:aggregationType']
#     journal = results['abstracts-retrieval-response']['coredata']['prism:publicationName']
#     volume = results['abstracts-retrieval-response']['coredata']['prism:volume']
#     articlenum = (results['abstracts-retrieval-response']['coredata'].get('prism:pageRange') or
#                   results['abstracts-retrieval-response']['coredata'].get('article-number'))
#     date = results['abstracts-retrieval-response']['coredata']['prism:coverDate']
#     doi = 'doi:' + results['abstracts-retrieval-response']['coredata']['prism:doi']
#     cites = int(results['abstracts-retrieval-response']['coredata']['citedby-count'])
    
#     return pubtype, f'{authors}, {title}, {journal}, {volume}, {articlenum}, ({date}). {doi} (cited {cites} times).\n'

In [12]:
# jehyun
def get_scopus_info(SCOPUS_ID):
    url = ("http://api.elsevier.com/content/abstract/scopus_id/"
          + sid
          + "?field=authors,affiliations,title,publicationName,volume,issueIdentifier,"
          + "prism:pageRange,coverDate,article-number,doi,citedby-count,prism:aggregationType")
    resp = requests.get(url,
                    headers={'Accept':'application/json',
                             'X-ELS-APIKey': APIKeys[-3]})
    results = json.loads(resp.text.encode('utf-8'))
    
    # authors full name for check
    authors_full = [f"{au['preferred-name']['ce:surname']}, {au['preferred-name']['ce:given-name']}" 
                    if 'preferred-name' in au.keys() else f"{au['ce:surname']}, {au['ce:given-name']}" 
                    for au in results['abstracts-retrieval-response']['authors']['author']]
    au_afs = [af["affiliation"] if ("affiliation" in af.keys()) else "none" for af in results['abstracts-retrieval-response']["authors"]["author"]]
    afids = []
    for au_af in au_afs:
        if isinstance(au_af, list):
            afids.append([af['@id'] for af in au_af])
        elif isinstance(au_af, dict):
            afids.append([au_af['@id']])

    authors = ', '.join([au['ce:indexed-name'] for au in results['abstracts-retrieval-response']['authors']['author']])
    title = results['abstracts-retrieval-response']['coredata']['dc:title'] if 'dc:title' in results['abstracts-retrieval-response']['coredata'].keys() else "none"
    pubtype = results['abstracts-retrieval-response']['coredata']['prism:aggregationType'] if 'prism:aggregationType' in results['abstracts-retrieval-response']['coredata'].keys() else "none"
    journal = results['abstracts-retrieval-response']['coredata']['prism:publicationName'] if 'prism:publicationName' in results['abstracts-retrieval-response']['coredata'].keys() else "none"
    volume = results['abstracts-retrieval-response']['coredata']['prism:volume'] if 'prism:volume' in results['abstracts-retrieval-response']['coredata'].keys() else "none"
    articlenum = (results['abstracts-retrieval-response']['coredata'].get('prism:pageRange') or
                  results['abstracts-retrieval-response']['coredata'].get('article-number'))
    date = results['abstracts-retrieval-response']['coredata']['prism:coverDate'] if 'prism:coverDate' in results['abstracts-retrieval-response']['coredata'].keys() else "none"
    doi = 'doi:' + results['abstracts-retrieval-response']['coredata']['prism:doi'] if 'prism:doi' in results['abstracts-retrieval-response']['coredata'].keys() else "none"
    cites = int(results['abstracts-retrieval-response']['coredata']['citedby-count']) if 'citedby-count' in results['abstracts-retrieval-response']['coredata'].keys() else "none"
        
    return pubtype, authors_full, afids, f'{date}: {authors}, {title}, {journal}, {volume}, {articlenum}, ({date}). {doi} (cited {cites} times).\n', results

In [13]:
df_my["my_scopusid"] = [[]] * df_my.shape[0]
df_my

,my_name,my_id,my_afs,my_afids,co_names,co_ids,my_scopusid
0,"Jehyun, Lee",15834698800,Seoul National University,60013682,"[Suess, Dieter, Schrefl, Thomas, Kyu, Hwan Oh,...","[7004076065, 7005780657, 6602521869, 35601590600]",[]
1,"Jehyun, Lee",15834698800,Technische Universität Wien,60018163,"[Suess, Dieter, Schrefl, Thomas, Kyu, Hwan Oh,...","[7004076065, 7005780657, 6602521869, 35601590600]",[]
2,"Lee, J.",16029235100,Technische Universität Wien,60018163,"[Fiedler, G., Fidler, J., Schrefl, T., Stamps,...","[15829081900, 35601590600, 7005780657, 7004916...",[]
3,"Lee, J.",24399437900,Technische Universität Wien,60018163,"[Schulze, C., Faustini, M., Schletter, H., Lut...","[35181915100, 57196395997, 36675826300, 350853...",[]
4,"Lee, J.",7601454858,Seoul National University,60013682,"[Neu, V., Schulze, C., Faustini, M., Makarov, ...","[6701391076, 35181915100, 57196395997, 1264713...",[]
5,"Lee, J. H.",7601451606,Seoul National University,60013682,"[Jang, S. H., Kim, Y. W., Kim, K. Y.]","[24821314300, 8099600200, 57203256286]",[]
6,"Lee, J. H.",7601462370,Korea Institute of Science and Technology,60025960,"[Jang, S. H., Kang, T., Kim, K. Y.]","[24821314300, 8953533300, 57203256286]",[]
7,"Lee, Je Hyun",16029235100,Korea Institute of Science and Technology,60025960,"[Kim, Dal Young, Kim, Jin Soo, Park, Bae Ho, L...","[7409765949, 55881881500, 7402834739, 56524680...",[]
8,"Lee, Jehyun",16029235100,Center of Semiconductor Research and Development,111165432,"[Kim, Sang Koog, Yoo, Myoung Woo, Lee, Jae Hye...","[7601587994, 35738330300, 57206731799, 5719974...",[]
9,"Lee, Jehyun",16029235100,Center of Semiconductor Research and Development,115225850,"[Kim, Bosung, Yoo, Myoung Woo, Kim, Sang Koog,...","[56621593100, 35738330300, 7601587994, 5620601...",[]


In [14]:
my_name_vs = df_my["my_name"].values
my_id_vs = df_my["my_id"].values
my_afids_vs = df_my["my_afids"].values

pub_no = 0
pubs = []

In [15]:
for i, (my_name_v, my_id_v, my_afid_v) in enumerate(zip(my_name_vs, my_id_vs, my_afids_vs)):
    # retrieve Scopus IDs
    print(f"# {i}")
    print(f"# ({my_name_v}, {my_id_v}, {my_afid_v})")
    scopus_ids_valid = []
    author_pubs_ = requests.get(f"http://api.elsevier.com/content/search/scopus?query=AU-ID({my_id_v})&field=dc:identifier&count=200",
                                   headers={'Accept':'application/json',
                                            'X-ELS-APIKey': APIKeys[-3]})
    author_pubs = author_pubs_.json()
    scopus_ids = [p['dc:identifier'] for p in author_pubs['search-results']["entry"]]
    print(f"# No. of scopus ids= {len(scopus_ids)}")

    # retrieve valid articles
    for j, sid in enumerate(scopus_ids):
        try:
            pub_type, authors_full, afids, pub_info, results = get_scopus_info(sid)
#             pub = f"{j}, {authors_full}, {results['abstracts-retrieval-response']['coredata']['dc:title']}"
            
            # test
#             pubs_test.append(pub)
            
            # original
            for my_name in my_names:
                flag_chk = len(np.where(my_name == np.array(authors_full))[0])
                if flag_chk > 0:
                    print(f"my_name = {my_name}, authors_full= {authors_full}")
            
                    name_idx = np.where(my_name == np.array(authors_full))[0][0]
                    name_afid = afids[name_idx]
                    if (my_afid_v in name_afid) and (pub_type == 'Journal'):
                        scopus_ids_valid.append(sid)
                        pubs.append(f"{pub_info}")
                        pub_no += 1
                        print("* " + pub_info)
        except Exception as err:
            print(err)
            pass

print(pubs)

# 0
# (Jehyun, Lee, 15834698800, 60013682)
# No. of scopus ids= 1
my_name = Jehyun, Lee, authors_full= ['Jehyun, Lee', 'Suess, Dieter', 'Schrefl, Thomas', 'Kyu, Hwan Oh', 'Fidler, Josef']
* 2006-10-01: Jehyun L., Suess D., Schrefl T., Kyu H.O., Fidler J., Contribution of local incoherency on Gilbert-damping, IEEE Transactions on Magnetics, 42, 3210-3212, (2006-10-01). doi:10.1109/TMAG.2006.880565 (cited 3 times).

# 1
# (Jehyun, Lee, 15834698800, 60018163)
# No. of scopus ids= 1
my_name = Jehyun, Lee, authors_full= ['Jehyun, Lee', 'Suess, Dieter', 'Schrefl, Thomas', 'Kyu, Hwan Oh', 'Fidler, Josef']
* 2006-10-01: Jehyun L., Suess D., Schrefl T., Kyu H.O., Fidler J., Contribution of local incoherency on Gilbert-damping, IEEE Transactions on Magnetics, 42, 3210-3212, (2006-10-01). doi:10.1109/TMAG.2006.880565 (cited 3 times).

# 2
# (Lee, J., 16029235100, 60018163)
# No. of scopus ids= 41
my_name = Lee, Jehyun, authors_full= ['Kim, Sang Koog', 'Yoo, Myoung Woo', 'Lee, Jehyun', 'Lee, Jae H

In [16]:
len(pubs)

97

In [21]:
len(set(pubs))

0

# test

In [29]:
#5
get_my_data("10.1063/1.2126124")

['Lee, J. H.',
 '7601451606',
 ['Seoul National University'],
 ['60013682'],
 ['Jang, S. H.', 'Kim, Y. W.', 'Kim, K. Y.'],
 ['24821314300', '8099600200', '57203256286']]

In [30]:
#6
get_my_data("10.1016/j.jmmm.2003.12.707")

['Lee, J. H.',
 '7601462370',
 ['Korea Institute of Science and Technology'],
 ['60025960'],
 ['Jang, S. H.', 'Kang, T.', 'Kim, K. Y.'],
 ['24821314300', '8953533300', '57203256286']]

In [26]:
my_id_v

'7601462370'

In [16]:
%%time

# test
start = 5

pub_no_test = 0
pubs_test = []

for i, (my_name_v, my_id_v, my_afid_v) in enumerate(zip(my_name_vs[start:start+1], my_id_vs[start:start+1], my_afids_vs[start:start+1]), start):
    # retrieve Scopus IDs
    print(f"# {i}")
    print(f"# ({my_name_v}, {my_id_v}, {my_afid_v})")
    scopus_ids_valid = []
    author_pubs_ = requests.get(f"http://api.elsevier.com/content/search/scopus?query=AU-ID({my_id_v})&field=dc:identifier&count=200",
                                   headers={'Accept':'application/json',
                                            'X-ELS-APIKey': APIKeys[-3]})
    author_pubs = author_pubs_.json()
    scopus_ids = [p['dc:identifier'] for p in author_pubs['search-results']["entry"]]
    print(f"# No. of scopus ids= {len(scopus_ids)}")

    # retrieve valid articles
    for j, sid in enumerate(scopus_ids):
        try:
            pub_type, authors_full, afids, pub_info, results = get_scopus_info(sid)
#             pub = f"{j}, {authors_full}, {results['abstracts-retrieval-response']['coredata']['dc:title']}"
            
            # test
#             pubs_test.append(pub)
            
            # original
            for my_name in my_names:
                flag_chk = len(np.where(my_name == np.array(authors_full))[0])
                if flag_chk > 0:
                    print(f"my_name = {my_name}, authors_full= {authors_full}")            
                    name_idxs = np.where(my_name == np.array(authors_full))[0]
                    for name_idx in name_idxs:
                        name_afid = afids[name_idx]
                        if (my_afid_v in name_afid) and (pub_type == 'Journal'):
                            scopus_ids_valid.append(sid)
                            pubs_test.append(f"{pub_info}")
                            pub_no_test += 1
                            print("* " + pub_info)
        except Exception as err:
            print(err)
            pass

print(pubs_test)

# 5
# (Lee, J. H., 7601451606, 60013682)
# No. of scopus ids= 200
my_name = Lee, J. H., authors_full= ['Lee, I. N.', 'Kim, K. H.', 'Lee, J. H.', 'Lee, D. H.', 'You, K. H.']
my_name = Lee, J., authors_full= ['Vandenbroucke, A.', 'Lee, J.', 'Spanoudaki, V.', 'Lau, F. W.Y.', 'Reynolds, P. D.', 'Levin, C. S.']
my_name = Lee, J. H., authors_full= ['Lee, D. U.', 'You, J. H.', 'Kim, T. W.', 'Lee, J. H.', 'Yoo, K. H.', 'Bae, S. B.', 'Lee, K. S.', 'Ram-Mohan, L. R.']
my_name = Lee, J., authors_full= ['Lee, J.', 'Sin, D. D.']
my_name = Lee, J. H., authors_full= ['Kim, Y. J.', 'Lee, J. H.', 'Seo, K. J.', 'Yoon, C. R.', 'Roh, E. S.', 'Cho, J. K.', 'Hattori, T.']
my_name = Lee, J. H., authors_full= ['Kim, D. K.', 'Lee, J. H.']
my_name = Lee, J., authors_full= ['Park, S. M.', 'Kim, H.', 'Min, S.', 'Park, J.', 'Lee, J. H.', 'Kil, H.', 'Paxton, L. J.', 'Su, S. Y.', 'Lee, J.', 'Min, K. W.']
my_name = Lee, J. H., authors_full= ['Park, S. M.', 'Kim, H.', 'Min, S.', 'Park, J.', 'Lee, J. H.', 'Kil, H.', 'P

In [20]:
results

{'abstracts-retrieval-response': {'coredata': {'citedby-count': '0',
   'prism:volume': '6055',
   'prism:coverDate': '2006-04-10',
   'dc:title': 'New method of zoom-convergence interlocked control in the moving parallel-axes style stereoscopic camera',
   'prism:aggregationType': 'Conference Proceeding',
   'prism:doi': '10.1117/12.642606',
   'article-number': '60551M',
   'prism:publicationName': 'Proceedings of SPIE - The International Society for Optical Engineering'},
  'authors': {'author': [{'ce:given-name': 'Junyong',
     'preferred-name': {'ce:given-name': 'Junyong',
      'ce:initials': 'J.',
      'ce:surname': 'Lee',
      'ce:indexed-name': 'Lee J.'},
     '@seq': '1',
     'ce:initials': 'J.',
     '@_fa': 'true',
     'affiliation': {'@id': '100539002',
      '@href': 'https://api.elsevier.com/content/affiliation/affiliation_id/100539002'},
     'ce:surname': 'Lee',
     '@auid': '8728004900',
     'author-url': 'https://api.elsevier.com/content/author/author_id/87280

In [23]:
my_id_v

'7601462370'

In [140]:
tmp = {'coredata': {'citedby-count': '5', 'prism:volume': '620', 'prism:pageRange': '165-169', 'prism:coverDate': '2016-12-01', 'dc:title': 'Sn microparticles made by plasma-induced dewetting', 'prism:aggregationType': 'Journal', 'prism:doi': '10.1016/j.tsf.2016.07.081', 'prism:publicationName': 'Thin Solid Films'}, 'authors': {'author': [{'preferred-name': {'ce:given-name': 'H. J.', 'ce:initials': 'H.J.', 'ce:surname': 'Choe', 'ce:indexed-name': 'Choe H.J.'}, '@seq': '1', 'ce:initials': 'H.J.', '@_fa': 'true', 'affiliation': {'@id': '60013682', '@href': 'https://api.elsevier.com/content/affiliation/affiliation_id/60013682'}, 'ce:surname': 'Choe', '@auid': '54887611800', 'author-url': 'https://api.elsevier.com/content/author/author_id/54887611800', 'ce:indexed-name': 'Choe H.J.'}, {'preferred-name': {'ce:given-name': 'S. H.', 'ce:initials': 'S.H.', 'ce:surname': 'Kwon', 'ce:indexed-name': 'Kwon S.H.'}, '@seq': '2', 'ce:initials': 'S.-H.', '@_fa': 'true', 'affiliation': {'@id': '60013682', '@href': 'https://api.elsevier.com/content/affiliation/affiliation_id/60013682'}, 'ce:degrees': 'Dr.', 'ce:surname': 'Kwon', '@auid': '57037648300', 'author-url': 'https://api.elsevier.com/content/author/author_id/57037648300', 'ce:indexed-name': 'Kwon S.-H.'}, {'preferred-name': {'ce:given-name': 'C.', 'ce:initials': 'C.', 'ce:surname': 'Choe', 'ce:indexed-name': 'Choe C.'}, '@seq': '3', 'ce:initials': 'C.', '@_fa': 'true', 'affiliation': {'@id': '60013682', '@href': 'https://api.elsevier.com/content/affiliation/affiliation_id/60013682'}, 'ce:surname': 'Choe', '@auid': '57214583376', 'author-url': 'https://api.elsevier.com/content/author/author_id/57214583376', 'ce:indexed-name': 'Choe C.'}, {'preferred-name': {'ce:given-name': 'J. J.', 'ce:initials': 'J.J.', 'ce:surname': 'Lee', 'ce:indexed-name': 'Lee J.J.'}, '@seq': '4', 'ce:initials': 'J.-J.', '@_fa': 'true', 'affiliation': {'@id': '60013682', '@href': 'https://api.elsevier.com/content/affiliation/affiliation_id/60013682'}, 'ce:degrees': 'Prof', 'ce:surname': 'Lee', '@auid': '7601454858', 'author-url': 'https://api.elsevier.com/content/author/author_id/7601454858', 'ce:indexed-name': 'Lee J.-J.'}, {'preferred-name': {'ce:given-name': 'C. H.', 'ce:initials': 'C.H.', 'ce:surname': 'Woo', 'ce:indexed-name': 'Woo C.H.'}, '@seq': '5', 'ce:initials': 'C.-H.', '@_fa': 'true', 'affiliation': {'@id': '60023075', '@href': 'https://api.elsevier.com/content/affiliation/affiliation_id/60023075'}, 'ce:degrees': 'Prof', 'ce:surname': 'Woo', '@auid': '57192168617', 'author-url': 'https://api.elsevier.com/content/author/author_id/57192168617', 'ce:indexed-name': 'Woo C.-H.'}]}}
tmp['coredata']['dc:title']

'Sn microparticles made by plasma-induced dewetting'

In [ ]:
afids = [['112806019'], ['60008069'], ['60158061'], ['60013682'], ['60008069', '60007746'], ['60018163'], ['60013682'], ['60158061'], ['112806019'], ['60008069']]


In [124]:
pubs_test.append(pub)
pubs_test

['Yu Y.-S., Han D.-S., Yoo M.-W., Lee K.-S., Choi Y.-S., Jung H., Lee J., Im M.-Y., Fischer P., Kim S.-K., Resonant amplification of vortex-core oscillations by coherent magnetic-field pulses, Scientific Reports, 3, 1301, (2013-03-01). doi:10.1038/srep01301 (cited 11 times).\n',
 'Yu Y.-S., Han D.-S., Yoo M.-W., Lee K.-S., Choi Y.-S., Jung H., Lee J., Im M.-Y., Fischer P., Kim S.-K., Resonant amplification of vortex-core oscillations by coherent magnetic-field pulses, Scientific Reports, 3, 1301, (2013-03-01). doi:10.1038/srep01301 (cited 11 times).\n']

In [113]:
for sid in scopus_ids[:6]:
    pub_type, authors_full, afids, pub_info, results = get_scopus_info(sid)
    print(sid, authors_full)

SCOPUS_ID:84999277963
SCOPUS_ID:84999277963 ['Choe, H. J.', 'Kwon, S. H.', 'Choe, C.', 'Lee, J. J.', 'Woo, C. H.']
SCOPUS_ID:84920729671
SCOPUS_ID:84920729671 ['Kwon, Soon Ho', 'Choe, Han Joo', 'Lee, Hyo Chang', 'Chung, Chin Wook', 'Lee, Jung Joong']
SCOPUS_ID:84957900024
SCOPUS_ID:84957900024 ['Kim, Sungman', 'Chun, Dongwon', 'Lee, Jungjoong', 'Jeung, Won Young']
SCOPUS_ID:84885443819
SCOPUS_ID:84885443819 ['Kwon, Soon Ho', 'Choe, Han Joo', 'Lee, Hyo Chang', 'Chung, Chin Wook', 'Lee, Jung Joong']
SCOPUS_ID:84879463257
SCOPUS_ID:84879463257 ['Choe, Han Joo', 'Kwon, Soon Ho', 'Lee, Jung Joong']
SCOPUS_ID:84875132108
# Neu V., Schulze C., Faustini M., Lee J., Makarov D., Suess D., Kim S.-K., Grosso D., Schultz L., Albrecht M. Probing the energy barriers and magnetization reversal processes of nanoperforated membrane based percolated media, [['112806019'], ['60008069'], ['60158061'], ['60013682'], ['60008069', '60007746'], ['60018163'], ['60013682'], ['60158061'], ['112806019'], ['6000806

In [115]:
pub_type, authors_full, afids, pub_info, results = get_scopus_info("SCOPUS_ID:84875132108")
print(authors_full)

SCOPUS_ID:84875132108
# Neu V., Schulze C., Faustini M., Lee J., Makarov D., Suess D., Kim S.-K., Grosso D., Schultz L., Albrecht M. Probing the energy barriers and magnetization reversal processes of nanoperforated membrane based percolated media, [['112806019'], ['60008069'], ['60158061'], ['60013682'], ['60008069', '60007746'], ['60018163'], ['60013682'], ['60158061'], ['112806019'], ['60008069']]
['Neu, V.', 'Schulze, C.', 'Faustini, M.', 'Lee, J.', 'Makarov, D.', 'Suess, D.', 'Kim, S. K.', 'Grosso, D.', 'Schultz, L.', 'Albrecht, M.']


In [126]:
# jehyun 2
def get_scopus_info(SCOPUS_ID):
    url = ("http://api.elsevier.com/content/abstract/scopus_id/"
          + sid
          + "?field=authors,affiliations,title,publicationName,volume,issueIdentifier,"
          + "prism:pageRange,coverDate,article-number,doi,citedby-count,prism:aggregationType")
    resp = requests.get(url,
                    headers={'Accept':'application/json',
                             'X-ELS-APIKey': APIKeys[-3]})
    results = json.loads(resp.text.encode('utf-8'))
    
    # authors full name for check
    authors_full = [f"{au['preferred-name']['ce:surname']}, {au['preferred-name']['ce:given-name']}" 
                    if 'preferred-name' in au.keys() else f"{au['ce:surname']}, {au['ce:given-name']}" 
                    for au in results['abstracts-retrieval-response']['authors']['author']]
    au_afs = [af["affiliation"] if ("affiliation" in af.keys()) else "none" for af in results['abstracts-retrieval-response']["authors"]["author"]]
    afids = []
    for au_af in au_afs:
        if isinstance(au_af, list):
            afids.append([af['@id'] for af in au_af])
        elif isinstance(au_af, dict):
            afids.append([au_af['@id']])

    authors = ', '.join([au['ce:indexed-name'] for au in results['abstracts-retrieval-response']['authors']['author']])
    title = results['abstracts-retrieval-response']['coredata']['dc:title'] if 'dc:title' in results['abstracts-retrieval-response']['coredata'].keys() else "none"
    pubtype = results['abstracts-retrieval-response']['coredata']['prism:aggregationType'] if 'prism:aggregationType' in results['abstracts-retrieval-response']['coredata'].keys() else "none"
    journal = results['abstracts-retrieval-response']['coredata']['prism:publicationName'] if 'prism:publicationName' in results['abstracts-retrieval-response']['coredata'].keys() else "none"
    volume = results['abstracts-retrieval-response']['coredata']['prism:volume'] if 'prism:volume' in results['abstracts-retrieval-response']['coredata'].keys() else "none"
    articlenum = (results['abstracts-retrieval-response']['coredata'].get('prism:pageRange') or
                  results['abstracts-retrieval-response']['coredata'].get('article-number'))
    date = results['abstracts-retrieval-response']['coredata']['prism:coverDate'] if 'prism:coverDate' in results['abstracts-retrieval-response']['coredata'].keys() else "none"
    doi = 'doi:' + results['abstracts-retrieval-response']['coredata']['prism:doi'] if 'prism:doi' in results['abstracts-retrieval-response']['coredata'].keys() else "none"
    cites = int(results['abstracts-retrieval-response']['coredata']['citedby-count']) if 'citedby-count' in results['abstracts-retrieval-response']['coredata'].keys() else "none"
        
    return pubtype, authors_full, afids, f'{authors}, {title}, {journal}, {volume}, {articlenum}, ({date}). {doi} (cited {cites} times).\n', results

In [64]:
[af["affiliation"] for af in results['abstracts-retrieval-response']["authors"]["author"]]

[{'@id': '60013682',
  '@href': 'https://api.elsevier.com/content/affiliation/affiliation_id/60013682'},
 {'@id': '60013682',
  '@href': 'https://api.elsevier.com/content/affiliation/affiliation_id/60013682'},
 {'@id': '60013682',
  '@href': 'https://api.elsevier.com/content/affiliation/affiliation_id/60013682'},
 {'@id': '60013682',
  '@href': 'https://api.elsevier.com/content/affiliation/affiliation_id/60013682'},
 {'@id': '60023075',
  '@href': 'https://api.elsevier.com/content/affiliation/affiliation_id/60023075'}]

In [51]:
results['abstracts-retrieval-response']['authors']['author'][0]['preferred-name']

{'ce:given-name': 'H. J.',
 'ce:initials': 'H.J.',
 'ce:surname': 'Choe',
 'ce:indexed-name': 'Choe H.J.'}

In [45]:
results

{'abstracts-retrieval-response': {'coredata': {'citedby-count': '5',
   'prism:volume': '620',
   'prism:pageRange': '165-169',
   'prism:coverDate': '2016-12-01',
   'dc:title': 'Sn microparticles made by plasma-induced dewetting',
   'prism:aggregationType': 'Journal',
   'prism:doi': '10.1016/j.tsf.2016.07.081',
   'prism:publicationName': 'Thin Solid Films'},
  'authors': {'author': [{'preferred-name': {'ce:given-name': 'H. J.',
      'ce:initials': 'H.J.',
      'ce:surname': 'Choe',
      'ce:indexed-name': 'Choe H.J.'},
     '@seq': '1',
     'ce:initials': 'H.J.',
     '@_fa': 'true',
     'affiliation': {'@id': '60013682',
      '@href': 'https://api.elsevier.com/content/affiliation/affiliation_id/60013682'},
     'ce:surname': 'Choe',
     '@auid': '54887611800',
     'author-url': 'https://api.elsevier.com/content/author/author_id/54887611800',
     'ce:indexed-name': 'Choe H.J.'},
    {'preferred-name': {'ce:given-name': 'S. H.',
      'ce:initials': 'S.H.',
      'ce:surna

In [ ]:
pub_no = 1
pubs = []
for sid in scopus_ids:
    try:
        pub_type, pub_info = get_scopus_info(sid)
        if pub_type == 'Journal':
            pubs.append(f"{pub_info}")
            pub_no += 1
    except:
        pass

print(pub_no)

In [29]:
# export to .docx file
document = Document()
document.add_heading(f"Jehyun Lee({len(np.unique(pubs))} publications)\n")

for pub in np.unique(pubs):
    document.add_paragraph(f'{pub}', style='List Number')
document.save(f'my_CV.docx')